In [38]:
import requests
import json
import pandas as pd

# API key
API_KEY = '2c6ede60b3f31387e0fe63701ed6b35a'
API_KEY = 'e9ff0215790837e4584dcc0af2456316'

# API endpoint
# https://api.elsevier.com/content/subject/scidir

# Example query

# req = requests.get('https://api.elsevier.com/content/subject/scidir?code=391&apiKey={API_KEY}')


# Headers with API key
headers = {
    'Accept': 'application/json',
    'X-ELS-APIKey': API_KEY
}

# Search for articles that have "tensile strength" in the abstract
# https://api.elsevier.com/content/search/scidir?query=abs(tensile%20strength)&apiKey={API_KEY}

"""
{
  "authors": "Crick AND Watson",
  "title": "Molecular Structure OR DNA",
  "qs": "NOT London"
}
"""

params = {
    'query': 'all("tensile strength")'
}

# req = requests.get('https://api.elsevier.com/content/search/sciencedirect', headers=headers, params=params) # funciona
req = requests.get('https://api.elsevier.com/content/search/scopus', headers=headers, params=params) # funciona

# Make a request to the API
# req = requests.get('https://api.elsevier.com/content/search/sciencedirect?query=heart+attack%20AND%20text(liver)', headers=headers)

# Convert the response to JSON
data = req.json()

# Print the JSON
print(json.dumps(data, indent=4, sort_keys=True))


{
    "search-results": {
        "entry": [
            {
                "@_fa": "true",
                "affiliation": [
                    {
                        "@_fa": "true",
                        "affiliation-city": "Nanjing",
                        "affiliation-country": "China",
                        "affilname": "Southeast University"
                    }
                ],
                "citedby-count": "0",
                "dc:creator": "Zhao P.",
                "dc:identifier": "SCOPUS_ID:85160431190",
                "dc:title": "Study on macro and micro shear strength of continuously graded and gap-graded sand",
                "eid": "2-s2.0-85160431190",
                "link": [
                    {
                        "@_fa": "true",
                        "@href": "https://api.elsevier.com/content/abstract/scopus_id/85160431190",
                        "@ref": "self"
                    },
                    {
                        "@_fa": "t

In [39]:
req.reason

'OK'

In [40]:
data

{'search-results': {'opensearch:totalResults': '355887',
  'opensearch:startIndex': '0',
  'opensearch:itemsPerPage': '25',
  'opensearch:Query': {'@role': 'request',
   '@searchTerms': 'all("tensile strength")',
   '@startPage': '0'},
  'link': [{'@_fa': 'true',
    '@ref': 'self',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=25&query=all%28%22tensile+strength%22%29',
    '@type': 'application/json'},
   {'@_fa': 'true',
    '@ref': 'first',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=25&query=all%28%22tensile+strength%22%29',
    '@type': 'application/json'},
   {'@_fa': 'true',
    '@ref': 'next',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=25&count=25&query=all%28%22tensile+strength%22%29',
    '@type': 'application/json'},
   {'@_fa': 'true',
    '@ref': 'last',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=4975&count=25&query=all%28%22tensile+strength%22%29',
    '@type': 'app

In [ ]:
papers = []

done = False
cursor = '*'
i = 0


while not done:

    # API endpoint URL
    url = f'https://api.elsevier.com/content/search/scopus?query=SUBJTERMS({asjc_code})&cursor={cursor}&count=25'
    # Logger 
    print(f'Getting data from i {i}...')

    # Send GET request to the API
    response = requests.get(url, headers=headers)

    # Check the response status code
    if response.status_code == 200:
        data = response.json()

        # Get cursor for the next page
        cursor = data['search-results']['cursor']['@next']

        # Check if the cursor is empty
        if cursor == '':
            done = True

        # Get the abstracts
        for entry in data['search-results']['entry']:

            try:

                # Get the abstract url
                abstract_url = entry["link"][0]["@href"]

                # Get the abstract
                abstract_request = requests.get(abstract_url, headers=headers)
                abstract = abstract_request.json()

                # Get the abstract text
                if not 'dc:description' in abstract['abstracts-retrieval-response']['coredata']:
                    print('No abstract')
                    continue

                abstract_text = abstract['abstracts-retrieval-response']['coredata']['dc:description']

                # Check if abstract has at least 1 number
                if any(char.isdigit() for char in abstract_text):

                    # If so, save it
                    abstract_text = abstract_text.replace('\n', ' ')

                    # Get the title
                    title = abstract['abstracts-retrieval-response']['coredata']['dc:title']

                    # Get the doi
                    if not 'prism:doi' in abstract['abstracts-retrieval-response']['coredata']:
                        print('No doi')
                        continue
                    doi = abstract['abstracts-retrieval-response']['coredata']['prism:doi']

                    # Create a dictionary with the data
                    paper = {
                        'title': title,
                        'abstract': abstract_text,
                        'doi': doi,
                        'asjc_code': asjc_code
                    }

                    # Append the dictionary to the list
                    papers.append(paper)

            except Exception as e:
                print('Error getting ENTRY')
                print(e)
            
    else:
        print('Error:', response.status_code)

    # Break after 16 pages, that is, 400 papers
    i += 1
    if i == 16:
        break


# Create a dataframe with the data
df = pd.DataFrame(papers)


In [41]:
papers = []

for entry in data['search-results']['entry']:

    # Print name of article
    print(entry['dc:title'])


    # Get the abstract url
    abstract_url = entry["link"][0]["@href"]

    # Get the abstract
    abstract_request = requests.get(abstract_url, headers=headers)
    abstract = abstract_request.json()

    if not 'abstracts-retrieval-response' in abstract:
        print('No abstract')
        continue

    # Get the abstract text
    if not 'dc:description' in abstract['abstracts-retrieval-response']['coredata']:
        print('No abstract')
        continue

    abstract_text = abstract['abstracts-retrieval-response']['coredata']['dc:description']

    # Check if abstract has at least 1 number
    if any(char.isdigit() for char in abstract_text):

        # If so, save it
        abstract_text = abstract_text.replace('\n', ' ')

        # Get the title
        title = abstract['abstracts-retrieval-response']['coredata']['dc:title']

        # Get the doi
        if not 'prism:doi' in abstract['abstracts-retrieval-response']['coredata']:
            print('No doi')
            continue
        doi = abstract['abstracts-retrieval-response']['coredata']['prism:doi']

        # Create a dictionary with the data
        paper = {
            'title': title,
            'abstract': abstract_text,
            'doi': doi,
        }

        # Append the dictionary to the list
        papers.append(paper)

Study on macro and micro shear strength of continuously graded and gap-graded sand
A method to determine the bonded-particle model parameters for simulation of ores
Influence of pore structure on thermal stress distribution inside coal particles during primary fragmentation
Preliminary Properties and Biodegradability of Bioplastics Derived from Mangrove Fruit (Avicennia lanata) and Glycerol
Achieving ultrahigh strength and ductility in high-entropy alloys via dual precipitation
Ipomoea pes-tigridis L. extract accelerates wound healing in Wistar albino rats in excision and incision models
Performance of ferrocement composites circular tanks as a new approach for RC tanks
Optimization of Butt-joint laser welding parameters for elimination of angular distortion using High-fidelity simulations and Machine learning
Potential for rupture before eruption at Campi Flegrei caldera, Southern Italy
Enhancement in mechanical and antimicrobial properties of epoxidized natural rubber via reactive bl

In [43]:
papers

[{'title': 'Preliminary Properties and Biodegradability of Bioplastics Derived from Mangrove Fruit (Avicennia lanata) and Glycerol',
  'abstract': "Bioplastics are made from starch that can be decomposed naturally to be an alternative to commercial plastics. The research objective was to know the effect of the composition of mangrove powder and glycerol on the characteristics of the synthesized bioplastics. This study also obtained the maximum composition of bioplastics made from mangrove powder by adding glycerol and chitosan. In this study, bioplastics were synthesized using starch as an optimizer of the mechanical properties of bioplastics, chitosan as reinforcement and water repellent, and glycerol as a plasticizer. The results of bioplastics were characterized by mechanical properties (tensile strength and elongation), hydrophobility, biodegradation, and functional groups. Based on the study's results, the optimum composition of bioplastics was found, namely mangrove 2 grams and g

In [44]:
df = pd.DataFrame(papers)

In [53]:
df

,title,abstract,doi,sentences
0,Preliminary Properties and Biodegradability of...,Bioplastics are made from starch that can be d...,10.33263/LIANBS124.147,"[Based on the study's results, the optimum com..."
1,Achieving ultrahigh strength and ductility in ...,The strength-ductility trade-off has been a lo...,10.1016/j.jmst.2023.05.021,[(Ni2Co2FeCr)96–xAl4Nbx (at.%) alloys strength...
2,Ipomoea pes-tigridis L. extract accelerates wo...,Ethnopharmacological relevance: An annual herb...,10.1016/j.jep.2023.116808,[Results: The maximum amount of polyphenolic c...
3,Performance of ferrocement composites circular...,Due to ferrocement high strength-to-weight rat...,10.1016/j.cscm.2023.e02228,[The investigational work includes testing twe...
4,Optimization of Butt-joint laser welding param...,The high temperature gradient and high cooling...,10.1016/j.optlastec.2023.109566,[It shows that SS316L tensile test specimens f...
5,Potential for rupture before eruption at Campi...,Volcanoes reawakening after long repose must r...,10.1038/s43247-023-00842-1,[A rupturing sequence has been developing acro...
6,Enhancement in mechanical and antimicrobial pr...,An epoxidized natural rubber (ENR) blend with ...,10.1038/s41598-023-36962-z,[An epoxidized natural rubber (ENR) blend with...
7,"A simple, fast, portable and selective system ...","In this work, an electrochemical method was de...",10.1016/j.talanta.2023.124810,[The sensor showed a 7 times higher oxidation ...
8,Does mechanical loading restore ligament biome...,Background: Mechanical loading is purported to...,10.1186/s12891-023-06653-x,[Methods: Electronic and supplemental searches...
9,Effect of laser shock peening on microstructur...,The effect of laser shock peening (LSP) on the...,10.1038/s41598-023-37060-w,"[After LSP, the microhardness of the cladding ..."


In [56]:
df.to_excel('sentences-tensile.xlsx')

In [46]:
!pip install openpyxl

  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [54]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
df["sentences"] = df["abstract"].apply(lambda x: x.split(". "))

def check_sentences(sentences: list) -> list:

    final_sentences = []
    for sentence in sentences:

        # Delete ( and ) from sentence
        sentence_temp = sentence.replace("(", "")
        sentence_temp = sentence_temp.replace(")", "")

        # If sentence has a %, return True
        if "%" in sentence:
            final_sentences.append(sentence)
            continue
        
        
        # Split sentence into words
        words = sentence_temp.split(" ")

        # If at least one word is a number, return True
        for word in words:
            if is_number(word):
                final_sentences.append(sentence)
                break

    return final_sentences
        
# If sentence doesn't have a number, remove it:
df["sentences"] = df["sentences"].apply(lambda x: check_sentences(x))


In [55]:
df

,title,abstract,doi,sentences
0,Preliminary Properties and Biodegradability of...,Bioplastics are made from starch that can be d...,10.33263/LIANBS124.147,"[Based on the study's results, the optimum com..."
1,Achieving ultrahigh strength and ductility in ...,The strength-ductility trade-off has been a lo...,10.1016/j.jmst.2023.05.021,[(Ni2Co2FeCr)96–xAl4Nbx (at.%) alloys strength...
2,Ipomoea pes-tigridis L. extract accelerates wo...,Ethnopharmacological relevance: An annual herb...,10.1016/j.jep.2023.116808,[Results: The maximum amount of polyphenolic c...
3,Performance of ferrocement composites circular...,Due to ferrocement high strength-to-weight rat...,10.1016/j.cscm.2023.e02228,[The investigational work includes testing twe...
4,Optimization of Butt-joint laser welding param...,The high temperature gradient and high cooling...,10.1016/j.optlastec.2023.109566,[It shows that SS316L tensile test specimens f...
5,Potential for rupture before eruption at Campi...,Volcanoes reawakening after long repose must r...,10.1038/s43247-023-00842-1,[A rupturing sequence has been developing acro...
6,Enhancement in mechanical and antimicrobial pr...,An epoxidized natural rubber (ENR) blend with ...,10.1038/s41598-023-36962-z,[An epoxidized natural rubber (ENR) blend with...
7,"A simple, fast, portable and selective system ...","In this work, an electrochemical method was de...",10.1016/j.talanta.2023.124810,[The sensor showed a 7 times higher oxidation ...
8,Does mechanical loading restore ligament biome...,Background: Mechanical loading is purported to...,10.1186/s12891-023-06653-x,[Methods: Electronic and supplemental searches...
9,Effect of laser shock peening on microstructur...,The effect of laser shock peening (LSP) on the...,10.1038/s41598-023-37060-w,"[After LSP, the microhardness of the cladding ..."


In [57]:
df.to_pickle('sentences-tensile.pkl')